# Electricity Price Prediction - Improved Version

Bu notebook, veri hazırlama ve **algo_test** fonksiyonu ile çoklu model karşılaştırması yapmaktadır.

## 1. Kütüphanelerin Yüklenmesi

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns",100)

from sklearn.linear_model import LinearRegression,SGDRegressor,Ridge,Lasso,ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor,AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
from sklearn.preprocessing import MinMaxScaler

%matplotlib inline

## 2. algo_test Fonksiyonu Tanımı

In [ ]:
def algo_test(x,y):
    L=LinearRegression()
    R=Ridge()
    Lass=Lasso()
    E=ElasticNet()
    sgd=SGDRegressor()
    ETR=ExtraTreeRegressor()
    GBR=GradientBoostingRegressor()
    kn=KNeighborsRegressor()
    ada=AdaBoostRegressor()
    dt=DecisionTreeRegressor()
    xgb=XGBRegressor()
    svr=SVR()
    mlp_regressor = MLPRegressor()

    algos=[L,R,Lass,E,sgd,ETR,GBR,ada,kn,dt,xgb,svr,mlp_regressor]
    algo_names=['Linear','Ridge','Lasso','ElasticNet','SGD','Extra Tree','Gradient Boosting',
                'AdaBoost','KNeighborsRegressor','Decision Tree','XGBRegressor','SVR','mlp_regressor']
    
    x=MinMaxScaler().fit_transform(x)
    x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=.20,random_state=42)
    
    r_squared= []
    rmse= []
    mae= []
    
    result=pd.DataFrame(columns=['R_Squared','RMSE','MAE'],index=algo_names)
    
    for algo in algos:
        p=algo.fit(x_train,y_train).predict(x_test)
        r_squared.append(r2_score(y_test,p))
        rmse.append(mean_squared_error(y_test,p)**.5)
        mae.append(mean_absolute_error(y_test,p))

    result.R_Squared=r_squared
    result.RMSE=rmse
    result.MAE=mae
    
    rtable=result.sort_values('R_Squared',ascending=False)
    return rtable

## 3. Veri Yükleme

In [ ]:
data = pd.read_csv("data/electricity.csv", low_memory=False)
print(f"Veri boyutu: {data.shape}")
data.head()

## 4. Veri Tipi Dönüşümleri

In [ ]:
# Object tipinde olan sayısal sütunları dönüştür
numeric_cols = ["ForecastWindProduction", "SystemLoadEA", "SMPEA", 
                "ORKTemperature", "ORKWindspeed", "CO2Intensity", 
                "ActualWindProduction", "SystemLoadEP2", "SMPEP2"]

for col in numeric_cols:
    data[col] = pd.to_numeric(data[col], errors='coerce')
    
print("✓ Veri tipleri dönüştürüldü")

In [ ]:
# Eksik değerleri kontrol et
print("Eksik değerler:")
print(data.isnull().sum())

In [ ]:
# Eksik değerleri kaldır
data = data.dropna()
print(f"✓ Temizlik sonrası veri boyutu: {data.shape}")

## 5. Özellik Seçimi ve X, y Tanımlama

In [ ]:
# Özellik sütunları
feature_columns = ["Day", "Month", "Year", "DayOfWeek", "WeekOfYear", "PeriodOfDay",
                   "ForecastWindProduction", "SystemLoadEA", "SMPEA", 
                   "ORKTemperature", "ORKWindspeed", "CO2Intensity", 
                   "ActualWindProduction", "SystemLoadEP2"]

X = data[feature_columns]
y = data["SMPEP2"]  # Hedef: Elektrik fiyatı

print(f"✓ Özellik sayısı: {len(feature_columns)}")
print(f"✓ Özellikler: {feature_columns}")
print(f"✓ X boyutu: {X.shape}")
print(f"✓ y boyutu: {y.shape}")

## 6. algo_test ile Tüm Modelleri Çalıştır

In [ ]:
results = algo_test(X, y)
results

## 7. Sonuç Görselleştirme

In [ ]:
plt.figure(figsize=(12, 8))
colors = ['green' if r > 0 else 'red' for r in results['R_Squared']]
plt.barh(results.index, results['R_Squared'], color=colors, alpha=0.7, edgecolor='black')
plt.axvline(x=0, color='black', linestyle='-', linewidth=1)
plt.xlabel('R² Score')
plt.title('Electricity Price Prediction - Model R² Karşılaştırması')
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

---

## Açıklama: Yapılan İşlemler

Bu notebook'ta aşağıdaki adımlar gerçekleştirilmiştir:

### 1. Veri Yükleme ve Ön İşleme
- **electricity.csv** dosyası yüklendi
- Object tipindeki sayısal sütunlar (`ForecastWindProduction`, `SystemLoadEA`, `SMPEA` vb.) `pd.to_numeric()` ile sayısal tipe dönüştürüldü
- Eksik değerler (`NaN`) `dropna()` ile kaldırıldı

### 2. Özellik Mühendisliği (Feature Engineering)
Modelleme için kullanılan özellikler:
- **Zaman özellikleri**: Day, Month, Year, DayOfWeek, WeekOfYear, PeriodOfDay
- **Enerji özellikleri**: ForecastWindProduction, ActualWindProduction, SystemLoadEA, SystemLoadEP2
- **Çevresel özellikler**: ORKTemperature, ORKWindspeed, CO2Intensity
- **Fiyat özellikleri**: SMPEA

### 3. Hedef Değişken
- **SMPEP2**: Elektrik fiyatı (System Marginal Price EP2)

### 4. algo_test Fonksiyonu
Aşağıdaki 13 regresyon modeli tek bir fonksiyon çağrısıyla test edildi:
- Linear Regression
- Ridge Regression
- Lasso Regression
- ElasticNet
- SGD Regressor
- Extra Tree Regressor
- Gradient Boosting Regressor
- AdaBoost Regressor
- K-Neighbors Regressor
- Decision Tree Regressor
- XGBoost Regressor
- Support Vector Regressor (SVR)
- MLP Regressor (Neural Network)

### 5. Değerlendirme Metrikleri
Her model için hesaplanan metrikler:
- **R² Score (R_Squared)**: Modelin açıklayıcılık gücü (1'e yakın olması iyidir)
- **RMSE**: Root Mean Squared Error (düşük olması iyidir)
- **MAE**: Mean Absolute Error (düşük olması iyidir)

### 6. Sonuçlar
Tüm modeller R² değerine göre sıralanarak karşılaştırmalı tablo ve görselleştirme ile sunulmuştur.